<a href="https://colab.research.google.com/github/UjjS/NLP/blob/master/NLP(Chat_BOT_using_json).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

26 documents
6 classes ['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']
48 unique lemmatized words ['a', 'age', 'am', 'anyone', 'are', 'buy', 'call', 'could', 'cya', 'day', 'do', 'eat', 'get', 'good', 'goodbye', 'guy', 'have', 'hello', 'hi', 'hour', 'how', 'i', 'id', 'is', 'later', 'leaving', 'like', 'menu', 'name', 'of', 'old', 'on', 'open', 'operation', 'reccommend', 'see', 'should', 'something', 'the', 'there', 'tim', 'to', 'up', 'what', 'whats', 'when', 'you', 'your']


In [ ]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
6/6 [==============================] - 0s 2ms/step - loss: 1.8594 - accuracy: 0.1923
Epoch 2/200
6/6 [==============================] - 0s 2ms/step - loss: 1.7420 - accuracy: 0.2692
Epoch 3/200
6/6 [==============================] - 0s 2ms/step - loss: 1.7405 - accuracy: 0.2692
Epoch 4/200
6/6 [==============================] - 0s 2ms/step - loss: 1.6172 - accuracy: 0.4615
Epoch 5/200
6/6 [==============================] - 0s 2ms/step - loss: 1.6422 - accuracy: 0.2692
Epoch 6/200
6/6 [==============================] - 0s 2ms/step - loss: 1.5635 - accuracy: 0.4231
Epoch 7/200
6/6 [==============================] - 0s 2ms/step - loss: 1.5717 - accuracy: 0.4231
Epoch 8/200
6/6 [==============================] - 0s 2ms/step - loss: 1.5780 - accuracy: 0.3846
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 1.3500 - accuracy: 0.6923
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 1.4643 - accuracy: 0.4615
Epoch 11/200
6/6 [===========

In [ ]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res
  

In [ ]:
while(True):
  msg=input("Vebed:")
  print(chatbot_response(msg))  

Good to see you again!
Hello!
We sell chocolate chip cookies for $2!
Good to see you again!
